<a href="https://colab.research.google.com/github/ryu622/gnn-counterattack-xai-v2/blob/exp%2Fadd-new-file/GNN_CounterAttack_Thesis_show.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GATにアテンションの可視化を組み込んだバージョン

前提：Permutation Feature Importance (PFI) を採用している 。PFIは、特定の入力特徴量がモデルの全体的な性能に与える影響度を測るモデル不可知論的な手法であり 、これにより「速度や角度といった特徴量が全体として重要である」というグローバルな解釈性を提供する 。一方で、特定の反例（例：なぜこの高速なカウンターアタックが失敗に終わったのか）に対して、「どのプレイヤー間の相互作用が決定打となったか」という、詳細でローカルな戦術的洞察を与えるには限界がある。   

In [ ]:
!pip install ipywidgets matplotlib pandas scikit-learn spektral tensorflow progressbar ml-insights

In [ ]:
!pip install torch torchvision torchaudio
!pip install torch_geometric numpy pandas scikit-learn

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader as PyGDataLoader
from torch_geometric.nn import GATConv, global_mean_pool
from torch_geometric.utils import dense_to_sparse
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
import numpy as np
import pickle
import logging
import sys
from tqdm import tqdm

# --- ロガーの設定 (元のコードから) ---
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
stdout_handler = logging.StreamHandler(sys.stdout)
if not logger.handlers:
    logger.addHandler(stdout_handler)

# --- 初期設定 (ウィジェット代替) ---
class MockDropdown:
    def __init__(self, value): self.value = value
class MockVBox:
    def __init__(self, children): self.children = children
class Checkbox:
    def __init__(self, value, description): self.value = value; self.description = description

adj_matrix = MockDropdown('normal')
edge_f_box = MockVBox([Checkbox(value=True, description='Player Distance')] * 6)
node_f_box = MockVBox([Checkbox(value=True, description='x coordinate')] * 12)



In [ ]:
# --- データロード (Google Driveから) ---
# 実行前にGoogle Driveのマウントが必要です
# file_path は '/content/drive/MyDrive/RAW_DATA/women.pkl' のような絶対パスを指す変数
from google.colab import drive
import os

# Google Driveをマウント
drive.mount('/content/drive')


file_path = '/content/drive/MyDrive/GNN_Football_Analysis/Raw_Data/women.pkl'


logger.info(f"Opening {file_path}...")
with open(file_path, 'rb') as handle:
    og_data = pickle.load(handle)
logger.info("データロード完了。")

# --- 2. 特徴量フィルタリング (セクション 2.6) ---
def filter_features(data, gender=None):
    edge_feature_idxs = [idx for idx, x in enumerate(edge_f_box.children) if x.value]
    node_feature_idxs = [idx for idx, x in enumerate(node_f_box.children) if x.value]
    global node_features
    # ノード特徴量の名前を定義 (ここでは簡易的にダミーで置き換える)
    #node_features = [f"Feature_{i}" for i in range(len(node_feature_idxs))]
    CORRECT_NODE_FEATURES = [
        "X Coordinate",           # 0
        "Y Coordinate",           # 1
        "vX",                     # 2
        "vY",                     # 3
        "Speed",                  # 4
        "Velocity Angle",         # 5
        "Distance to Goal",       # 6
        "Angle with Goal",        # 7
        "Distance to Ball",       # 8
        "Angle with Ball",        # 9
        "Time in Possession",     # 10
        "Is Ball Carrier"         # 11
    ]
    # 修正: ダミー名の代わりに、インデックスに基づいて正しい特徴量名を取得
    node_features = [CORRECT_NODE_FEATURES[i] for i in node_feature_idxs]
    mat_type = adj_matrix.value
    data[mat_type]['e'] = [x[:, edge_feature_idxs] for x in data[mat_type]['e']]
    data[mat_type]['x'] = [x[:, node_feature_idxs] for x in data[mat_type]['x']]
    return data

data = filter_features(og_data.copy())
print("特徴量のフィルタリング完了。")

In [ ]:
# --- 3. PyGデータセットクラスの定義と変換 ---
class PyG_CounterDataset(Dataset):
    def __init__(self, data, matrix_type, root=None, transform=None, pre_transform=None):
        self.raw_data = data
        self.matrix_type = matrix_type
        self._data_list = self.process_data()
        super().__init__(root, transform, pre_transform)

    def process_data(self):
        data_mat = self.raw_data[self.matrix_type]
        data_list = []

        for x_np, a_np, e_np, y_np in tqdm(zip(
            data_mat['x'], data_mat['a'], data_mat['e'], self.raw_data['binary']
        ), total=len(data_mat['x']), desc="Converting to PyG Data"):

            try:
                if x_np.shape[0] == 0: continue

                # 疎行列の修正: SciPyの疎行列を密行列に変換
                if hasattr(a_np, 'todense'): a_np = a_np.todense()

                # テンソルへの変換
                x = torch.tensor(x_np, dtype=torch.float)
                a = torch.tensor(a_np, dtype=torch.float)
                e = torch.tensor(e_np, dtype=torch.float)
                y = torch.tensor(y_np, dtype=torch.float).view(-1, 1)

                edge_index, _ = dense_to_sparse(a)

                if edge_index.numel() == 0: continue

                # エッジ特徴量の整合性チェック
                edge_attr = e if e.size(0) == edge_index.size(1) else None

                data_list.append(Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y))
            except Exception:
                continue

        return data_list

    def len(self): return len(self._data_list)
    def get(self, idx): return self._data_list[idx]

dataset_pyg = PyG_CounterDataset(data=data, matrix_type='normal')
print(f" PyTorch Geometricデータセットの変換が完了しました。全サンプル数: {len(dataset_pyg)}")

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv, global_mean_pool
import torch
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

# --- 1. PyG_GNN_Attn クラスの定義 ---
# PyG_GNN_Attn は PyG_GNN と同じアーキテクチャですが、アテンション値を返す機能を追加しています。
# ハイパーパラメータの設定 (前のセルで定義された変数を使用)
CHANNELS = 32
LAYERS = 3
ATTN_HEADS = 4
N_OUT = 1

class PyG_GNN_Attn(nn.Module):
    def __init__(self, in_channels, hidden_channels, num_layers, num_heads, out_channels):
        super(PyG_GNN_Attn, self).__init__()

        self.conv1 = GATConv(in_channels, hidden_channels, heads=num_heads, dropout=0.5, concat=True)
        self.convs = nn.ModuleList([
            GATConv(hidden_channels * num_heads, hidden_channels, heads=num_heads, dropout=0.5, concat=True)
            for _ in range(num_layers - 1)
        ])
        self.conv_out = GATConv(hidden_channels * num_heads, hidden_channels, heads=1, dropout=0.5, concat=False)

        self.dense1 = nn.Linear(hidden_channels, hidden_channels)
        self.dense_out = nn.Linear(hidden_channels, out_channels)

    def forward(self, data, return_attn=False):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        attn_weights = None

        # 最初のGAT層でのみアテンションを抽出
        if return_attn:
            x, (edge_index, attn_weights) = self.conv1(x, edge_index, return_attention_weights=True)
        else:
            x = self.conv1(x, edge_index)

        x = F.elu(x)

        for conv in self.convs:
            x = F.elu(conv(x, edge_index))

        x = F.elu(self.conv_out(x, edge_index))
        x = global_mean_pool(x, batch)

        x = F.relu(self.dense1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        output = torch.sigmoid(self.dense_out(x))

        if return_attn:
            return output, edge_index, attn_weights.squeeze()

        return output

# --- 2. モデルのインスタンス化と重みコピー ---
# model_pyg (訓練済みモデル) が既にメモリに存在することを前提とします。
model_attn = PyG_GNN_Attn(
    in_channels=dataset_pyg.num_node_features, hidden_channels=CHANNELS, num_layers=LAYERS,
    num_heads=ATTN_HEADS, out_channels=N_OUT
)
model_attn.load_state_dict(model_pyg.state_dict())
model_attn.eval()

# --- 3. アテンション抽出のためのデータ準備 ---
# dataset_te_pyg は前のステップで定義されたテストデータセットです。
sample_index = 0
single_frame = dataset_te_pyg[sample_index]
single_data = single_frame.to(next(model_attn.parameters()).device)

# --- 4. アテンション値の抽出を実行 ---
with torch.no_grad():
    prediction, edge_index_attn, weights_attn = model_attn(single_data, return_attn=True)

print("アテンション抽出の準備完了。")

In [ ]:
# --- メイン訓練ループの実行 ---
print("\n PyTorch GATモデルの訓練を開始します...")
for epoch in range(1, EPOCHS + 1):
    loss = train_pyg(model_pyg, loader_tr_pyg, optimizer_pyg, criterion)
    print(f"--- Epoch {epoch}/{EPOCHS} --- Loss: {loss:.4f}")

print("PyTorch GATモデルの訓練が完了しました。")

In [ ]:
#評価関数の定義

import torch
from sklearn.metrics import roc_auc_score, accuracy_score
import numpy as np

def evaluate_pyg(model, loader):
    """
    テストデータローダーを使用してモデルを評価し、AUCとAccuracyを計算する。
    """
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for data in loader:
            out = model(data)
            all_preds.append(out.cpu().numpy())
            all_labels.append(data.y.cpu().numpy())

    preds = np.concatenate(all_preds).flatten()
    labels = np.concatenate(all_labels).flatten()

    try:
        auc = roc_auc_score(labels, preds)
    except ValueError:
        auc = np.nan
        print(" Warning: テストセットに単一クラスのみ含まれるため、AUCは未定義です。")

    predicted_classes = (preds > 0.5).astype(int)
    accuracy = accuracy_score(labels, predicted_classes)

    return auc, accuracy

print("評価関数 'evaluate_pyg' が定義されました。")

In [ ]:
# 評価の実行

auc_te, acc_te = evaluate_pyg(model_pyg, loader_te_pyg_clean)

print(f"\n--- 最終テスト結果 (PyTorch GAT) ---")
print(f"Test AUC: {auc_te:.4f}")
print(f"Test Accuracy: {acc_te:.4f}")

In [ ]:
#キャリブレーション
from sklearn.calibration import calibration_curve
import matplotlib.pyplot as plt

# --- キャリブレーションデータの取得 ---
def get_calibration_data(model, loader, n_bins=10):
    """モデルの予測確率と真のラベルを取得し、キャリブレーション曲線のデータを計算する"""
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for data in loader:
            out = model(data)
            all_preds.append(out.cpu().numpy())
            all_labels.append(data.y.cpu().numpy())

    preds = np.concatenate(all_preds).flatten()
    labels = np.concatenate(all_labels).flatten()

    # scikit-learnのcalibration_curveを使用してデータを計算
    # prob_true: 各ビンの実際の正答率 (y軸)
    # prob_pred: 各ビンの平均予測確率 (x軸)
    prob_true, prob_pred = calibration_curve(
        y_true=labels,
        y_prob=preds,
        n_bins=n_bins,
        strategy='uniform' # 均一な幅のビンを使用
    )

    return prob_true, prob_pred, preds, labels

# --- 実行と描画 ---
prob_true, prob_pred, preds_te, labels_te = get_calibration_data(model_pyg, loader_te_pyg_clean)

plt.figure(figsize=(8, 8))
# モデルのキャリブレーション曲線
plt.plot(prob_pred, prob_true, marker='o', label='PyTorch GAT Model')
# 理想的なキャリブレーション (対角線)
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Perfectly Calibrated')

plt.xlabel('Average Predicted Probability (Confidence)')
plt.ylabel('Fraction of Positives (Accuracy)')
plt.title('Calibration Curve on Test Set')
plt.legend()
plt.grid(True)
plt.show()

print("キャリブレーション曲線の描画が完了しました。")
# 理想的な対角線に近ければ近いほど、モデルの予測は「正直」であると評価できます。

In [ ]:
#PFI
# --- 特徴量重要度の計算 ---
# 特徴量の名前は filter_features 関数で定義されたグローバル変数 node_features を使用します
global node_features # 以前のステップで定義されています

def calculate_permutation_importance(model, loader, features_list, n_repeats=5):
    """テストデータセット上で、各特徴量のパーミュテーション重要度 (AUCの低下) を計算する"""

    # 1. ベースラインAUCの計算
    auc, _ = evaluate_pyg(model, loader)
    baseline_auc = auc

    importance = {}

    # 2. 各特徴量に対して重要度を測定
    for idx, feature_name in enumerate(features_list):
        auc_decreases = []

        for _ in range(n_repeats):
            # 2.1. テストローダー内の全グラフをパーミュテーション処理
            permuted_data_list = []
            for batch_data in loader:
                # バッチ内のノード特徴量 x のコピーを作成
                x_permuted = batch_data.x.clone()

                # 特定の特徴量列 (idx) をランダムに入れ替え (パーミュテーション)
                # バッチ全体でランダムに行うのが一般的
                perm_indices = torch.randperm(x_permuted.size(0))
                x_permuted[:, idx] = x_permuted[perm_indices, idx]

                # 新しいxでデータオブジェクトを再構築 (バッチ構造を維持)
                permuted_batch = batch_data.clone()
                permuted_batch.x = x_permuted
                permuted_data_list.append(permuted_batch)

            # 2.2. パーミュテーション後のAUCを評価
            # Note: PyGDataLoaderはリストを受け取るため、ここで新しいDataLoaderを一時的に使用します
            permuted_loader = PyGDataLoader(permuted_data_list, batch_size=loader.batch_size, shuffle=False)
            permuted_auc, _ = evaluate_pyg(model, permuted_loader)
            auc_decreases.append(baseline_auc - permuted_auc)

        # 2.3. 平均低下量を格納
        importance[feature_name] = np.mean(auc_decreases)

    return importance

# --- 実行と描画 ---

# 注意: n_repeatsが多いほど精度が上がりますが、計算時間がかかります。
importance_results = calculate_permutation_importance(model_pyg, loader_te_pyg_clean, node_features, n_repeats=1) # まずは n_repeats=1 で試します

# 結果をソートして表示
sorted_importance = sorted(importance_results.items(), key=lambda item: item[1], reverse=True)

print("\n--- 特徴量重要度 (AUC低下量) ---")
for feature, decrease in sorted_importance:
    print(f"[{feature: <20}] : {decrease:.5f}")

# 棒グラフで視覚化
features = [item[0] for item in sorted_importance]
decreases = [item[1] for item in sorted_importance]

plt.figure(figsize=(10, 6))
plt.barh(features, decreases, color='skyblue')
plt.xlabel('AUC Decrease after Permutation')
plt.title('Permutation Feature Importance (PyTorch GAT)')
plt.gca().invert_yaxis() # 最も重要な特徴量を上に表示
plt.grid(axis='x', linestyle='--')
plt.show()

print("特徴量重要度の計算と描画が完了しました。")

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import torch

# --- 1. アテンション抽出のための変数定義 (前ステップで抽出済みを想定) ---
# single_data: テストセットから取得した単一のDataオブジェクト
# prediction: モデルの予測確率 (例: 0.48)
# edge_index_attn: エッジの接続リスト
# weights_attn: 抽出されたアテンション重み (NumPy配列に変換済み)

# --- 2. NetworkXグラフの構築と可視化 ---

# 1. テンソルをCPU上のNumPy配列に変換
# ノード特徴 (選手の座標) - x:0列目, y:1列目と仮定
x_coords = single_data.x[:, 0].cpu().numpy()
y_coords = single_data.x[:, 1].cpu().numpy()
edge_list = edge_index_attn.cpu().numpy().T
attention_weights = weights_attn.cpu().numpy()

# 2. NetworkXグラフの構築
G = nx.Graph()
num_nodes = len(x_coords)
G.add_nodes_from(range(num_nodes))
pos = {i: (x_coords[i], y_coords[i]) for i in range(num_nodes)} # ノード座標

# 3. エッジの重み付けと正規化 (描画の太さ用)
min_w = attention_weights.min()
max_w = attention_weights.max()
# NumPy配列として計算。描画の太さ (1〜11の範囲) に調整
# ゼロ割を避けるため微小値 (1e-6) を追加
normalized_weights_np = 10 * (attention_weights - min_w) / (max_w - min_w + 1e-6) + 1

# --- 修正後のグラフ構築部分 ---

# --- 修正後のグラフ構築部分 ---

# 4. グラフにエッジと重みを追加 (型変換エラー回避のためfloatに明示変換)
for i, (u, v) in enumerate(edge_list):

    # 🚨 最終修正点: NumPyのfloat型を強制的に経由させる
    try:
        # NumPyのfloat64型に変換し、その値をPythonのfloatとして取得する
        # 配列が残る場合でも、この強制変換で処理できるようになる可能性が高い
        weight_val = float(np.float64(normalized_weights_np[i]))

    except Exception:
        # 上記で失敗する場合、以前のコードの最も安全な方法を試みる
        weight_val = float(normalized_weights_np[i].flatten()[0])

    G.add_edge(u, v, weight=weight_val)

# 描画コマンドの実行
# --- 5. 可視化の実行コード ---

# 1. グラフの構築と重み付け (前のコードブロックで実行されたものと同じロジックを使用)
# x_coords, y_coords, prediction, edge_list, G, pos, num_nodes は既に定義済み

# エッジの太さを取得
# 🚨 ここでエラーが出ないはず：Gに格納された値は、前のループでfloat型に強制変換されているため
edge_widths = [G[u][v]['weight'] for u, v in G.edges()]

# 可視化の実行
plt.figure(figsize=(12, 8))
plt.title(f"GAT Local Attention Visualization (Prediction: {prediction.item():.2f})")

# ノード情報 (以前の定義を使用)
num_players = 22
num_players_per_team = 11
node_colors = ['red'] * num_players_per_team + ['blue'] * num_players_per_team + ['yellow']
node_sizes = [300] * num_players + [500]

# 描画コマンド
nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, label=True)
nx.draw_networkx_edges(G, pos, width=edge_widths, alpha=0.7, edge_color='black')
nx.draw_networkx_labels(G, pos, font_size=8)

plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.show()

print("✅ アテンションの可視化描画が完了しました。")

AIによる分析：これは、あなたが PyTorch GAT (グラフアテンションネットワーク) モデルで実行した、特定のカウンターアタックフレームにおけるローカルアテンションの可視化結果です。このグラフは、モデルが予測確率 0.48 (失敗がわずかに優勢) を導き出すために、ピッチ上のどの選手間の相互作用に最も「注目」したかを視覚的に示しています。⚽ グラフの構成要素と初期分析1. ノード（選手とボール）の配置ノード: 赤丸（攻撃側）、青丸（守備側）、黄色（ボール、ID 22）。配置: ボール（ID 22）は、ピッチの中央やや下（低いY座標）に位置しており、攻撃側と守備側の選手がその周辺に密集していることがわかります。ノードID: ノードにはID (0から22) が割り振られており、特定の選手やボールを識別できます。2. アテンション（注目の強さ）エッジの太さ: GATモデルが計算したアテンション係数 ($\alpha_{ij}$) の強さを表します。線が太いほど、その選手間の情報交換が予測に重要であったことを意味します。2. 🔍 アテンションの主要なパターン分析このフレームでは、予測確率が 0.48 と、成功と失敗が拮抗しています。これは、モデルが**「失敗につながる守備の相互作用」と「成功につながる攻撃の相互作用」**の両方に注目していることを示唆します。🥇 最も強いアテンション（太いエッジ）グラフの中央、ボールの近くに、最も太いエッジが集中しています。ノード 22 (ボール) と周辺選手間の相互作用:ボールと攻撃側選手（例: ID 0, 1, 3, 4）の間、およびボールと守備側選手（例: ID 7, 13, 14, 15）の間で、非常に太いエッジが見られます。解釈: モデルは、ボールを巡る瞬間の攻防、特にボール保持者とマーク役の選手の足元の状況に最大の注意を払っています。守備選手間の協力 (青-青):ノード 13と15、13と14など、守備側の選手同士を結ぶ太いエッジが多数確認できます。解釈: 守備側が密集地帯で効果的に連携を取り、スペースを埋めようとしている相互作用が、モデルの予測（失敗の可能性）に重要であると判断されています。🥈 重要な攻撃的相互作用攻撃選手間の連携 (赤-赤):攻撃側の選手（例: ID 0とID 4、ID 1とID 4）を結ぶ太いエッジが見られます。解釈: これは、ボールの近くでパスの選択肢を作り出したり、次の展開の準備をしたりといった、攻撃側の局所的な連携がモデルにとって重要であることを示します。🥉 周辺のアテンション遠い選手の低アテンション:ピッチのサイドにいる選手（例: 攻撃側 ID 10 や守備側 ID 21）を結ぶエッジは、多くが細いままです。解釈: モデルは、この決定的な瞬間に遠い位置の選手の絶対的な位置や動きは無視し、ボール周辺の密集地帯での相互作用に焦点を絞っていることが裏付けられます。3. 🎯 結論この可視化は、GATモデルがバイナリ予測を行う際、論理的かつ戦術的に適切な判断を下していることを示しています。モデルは、広大なピッチ全体ではなく、ボール周辺の狭い範囲における、複数の選手間の同時発生的な動きやマーク（守備の連携、攻撃の選択肢）に最も注意を払っています。

成功確率が最も高いフレームを抽出してみる

In [ ]:
# --- 1. 成功確率が最も高いサンプルを見つける ---
model_attn.eval()
max_pred = -1.0
best_index = -1

with torch.no_grad():
    for idx, data_frame in enumerate(dataset_te_pyg):
        # モデルと同じデバイスに移動
        data_frame = data_frame.to(next(model_attn.parameters()).device)

        # 予測確率の取得
        prediction_prob = model_attn(data_frame).item()

        if prediction_prob > max_pred:
            max_pred = prediction_prob
            best_index = idx

print(f"✅ 最も高い予測確率 {max_pred:.4f} を持つフレームのインデックス: {best_index}")

In [ ]:
# --- 2. 特定フレームのアテンション抽出と可視化 ---

# 成功フレームのデータ取得
single_frame = dataset_te_pyg[best_index]
single_data = single_frame.to(next(model_attn.parameters()).device)

# アテンション値の抽出
with torch.no_grad():
    prediction, edge_index_attn, weights_attn = model_attn(single_data, return_attn=True)

print(f"予測確率: {prediction.item():.2f}")

# 3. NetworkXグラフの構築と可視化 (以前のコードを使用)
x_coords = single_data.x[:, 0].cpu().numpy()
y_coords = single_data.x[:, 1].cpu().numpy()
edge_list = edge_index_attn.cpu().numpy().T
attention_weights = weights_attn.cpu().numpy()

G = nx.Graph()
num_nodes = len(x_coords)
G.add_nodes_from(range(num_nodes))
pos = {i: (x_coords[i], y_coords[i]) for i in range(num_nodes)}

# エッジの重み付けと正規化
min_w = attention_weights.min()
max_w = attention_weights.max()
normalized_weights = 10 * (attention_weights - min_w) / (max_w - min_w + 1e-6) + 1

for i, (u, v) in enumerate(edge_list):
    try:
        weight_val = float(normalized_weights[i].squeeze().item())
    except Exception:
        weight_val = float(normalized_weights[i].flatten()[0])

    G.add_edge(u, v, weight=weight_val)

# 描画コマンド
plt.figure(figsize=(12, 8))
plt.title(f"GAT Local Attention Visualization (SUCCESS Prediction: {prediction.item():.2f})")

num_players = 22
num_players_per_team = 11
node_colors = ['red'] * num_players_per_team + ['blue'] * num_players_per_team + ['yellow']
node_sizes = [300] * num_players + [500]

edge_widths = [G[u][v]['weight'] for u, v in G.edges()]

nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, label=True)
nx.draw_networkx_edges(G, pos, width=edge_widths, alpha=0.7, edge_color='black')
nx.draw_networkx_labels(G, pos, font_size=8)

plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.show()

AIによる分析： 成功フレーム (Prediction: 0.98) の詳細分析1. 🥇 決定的な位置：ゴール前への侵入集中位置の特定: 選手とボールの配置は、ピッチの**右サイド、相手ゴールに近いエリア（高い X 座標）**に極めて集中しています。これは、カウンターアタックが最終局面に入り、得点に最も近い状況にあることを示しています。ボールの位置: ボール（ノード 12、黄色）は、密集地帯の最前線に位置しており、すぐにシュートまたは最後のパスが出せる状況にあることが示唆されます。2. 🧠 アテンションの焦点：攻撃側の連携と優位性このフレームでモデルが最も強く注目しているのは、攻撃側の連携とボールへの集中です。アテンションの種類観察されるパターンモデルの解釈（成功の根拠）ボールと攻撃側 (黄-赤)ノード 12 (ボール) は、ノード 1, 4, 6, 9 など複数の攻撃側選手と非常に太いエッジで接続されています。モデルは、ボール周辺に複数のパスコースやシュートの選択肢が同時に存在している状況を認識しています。これは、守備側がマークを絞りきれない状態であり、成功を確信しています。攻撃側同士 (赤-赤)攻撃側選手同士（例：1-4、5-9、6-8）の間でも太いエッジが多く見られます。これは、攻撃側の選手がお互いの動きを理解し合い、守備ラインの裏や隙間を突くための効果的な動き出しやポジショニングを実行していることに注目していることを示します。遠いノードノード 10 のように、プレーの中心から離れたノードからのエッジは細く、モデルはほとんど無視しています。ローカルアテンションの有効性: モデルは、遠方の選手の動きではなく、得点につながる密集地帯の相互作用に焦点を絞ることで、予測精度を高めています。3. 🛡️ 守備側の対応（相対的な軽視）成功フレームにおいて、モデルは守備側の努力を認識しつつも、攻撃側の脅威度を上回るとは見ていません。青-青のエッジ: 守備側選手（青）同士を結ぶエッジも太いものがありますが、その太さはボールと攻撃側の太いエッジに比べて支配的ではありません。解釈: 守備側は密集し、組織的な対応を試みていますが、GATモデルは、守備側の努力よりも攻撃側の決定的な動きの方が上回っていると判断しています。つまり、守備側が「スペースを埋める」ことよりも、攻撃側が「決定的なスペースを生み出す」ことにアテンションが強く向いています。結論GATモデルは、カウンターアタックの成功を予測する際、**「ゴール前での攻撃側の数的優位性、組織的な連携、そしてシュート/パスの実行可能性」**という、戦術的に正しい決定要因に集中的に注目していることがわかります。成功の瞬間のモデルの「視線」は、ボールとそれを囲む攻撃側の選手間の関係性にロックされていると言えます。

成功と失敗の違い
GATローカルアテンションの分析1. 🥇 成功予測フレーム (Prediction: 0.98) の分析グラフimage_6f2b52.png (予測: 0.98)状況モデルはカウンターアタックがほぼ確実に成功すると予測しています。ノード配置選手とボールがピッチのゴールに近い右側に集中。ノード 12 (ボール) が相手ゴール前、攻撃側選手（赤）に囲まれています。主要アテンション（太いエッジ）🔴 ボールと攻撃側の接続: ボール（12）と攻撃側選手（1, 4, 9, 6）の間で、非常に太いエッジが集中しています。解釈モデルは、ゴール前での攻撃側の決定的なパス交換や、シュート体勢にある選手の動きに最大の注意を払っています。守備側（青）の選手がゴールを守るために密集していますが、モデルは攻撃側の優位性（太い赤-黄のエッジ）に注目し、予測を確定しています。2. 🥈 失敗予測フレーム (Prediction: 0.48) の分析グラフimage_6f363b.png (予測: 0.48)状況モデルは成功と失敗が拮抗している状態、または失敗がわずかに優勢と予測しています。ノード配置選手とボールがピッチの中央下部に密集しており、ゴールから遠い位置にあります。主要アテンション（太いエッジ）🔵 守備側の連携: 守備側選手（青）同士（例: 4と13, 14, 16）を結ぶエッジが太く、特にボール（22）の周囲で強いです。解釈モデルは、攻撃側がゴールへ向かう前に、守備側が組織的にスペースを埋め、ボールの出口を塞ぐ相互作用に注目しています。太い青-青のエッジは、守備側の効果的なブロックまたはマークが、攻撃を停滞させている（成功確率を下げている）根拠であると示唆しています。3. 🎯 GATが学習した戦術的洞察これら2つのグラフの比較から、GATモデルは以下の戦術的違いを判断基準にしていることがわかります。判断基準成功フレーム (0.98)失敗フレーム (0.48)注目の中心ボールと攻撃側の優位な選手 (ゴール前)ボールと守備側の連携線 (中央密集地)決定的な相互作用赤-黄 (シュート/パスコースが開いている)青-青 (守備側のスペースカバー)GATの結論優位な位置での攻撃的連携が成功を確定させる。守備側の効果的な協力が攻撃を停滞させ、失敗に傾かせる。

In [ ]:
# --- 守備側のみの内部連携 (Blue-Blue) ---


# ノードIDの範囲を定義
DEFENSE_START_ID = 11
DEFENSE_END_ID = 21

defense_edges = []
defense_widths = []

# 🚨 修正点: edge_widths[i] ではなく、正規化された重みリストの i 番目の要素を直接参照する
# 注: attention_weights または normalized_weights_np がリスト i の長さを持っていると仮定
for i, (u, v) in enumerate(edge_list):

    is_defense_u = (DEFENSE_START_ID <= u <= DEFENSE_END_ID)
    is_defense_v = (DEFENSE_START_ID <= v <= DEFENSE_END_ID)

    if is_defense_u and is_defense_v:
        defense_edges.append((u, v))
        # 参照するのは元の正規化された重みリストの i 番目の要素
        # weight_val は float() に変換済みであるため、ここでは float を使用します
        defense_widths.append(G[u][v]['weight'])
        # もし G[u][v]['weight'] が float でない場合は、G.edges(data=True) を使う必要がありますが、
        # ここでは G に格納されている 'weight' 属性を直接参照します。

# G.edges() を使ってエッジと重みをループで取得するのが最も確実です
# ただし、元のロジックに従い、フィルタリングには edge_list を使用します。

# 🚨 修正後のロジック (Gに格納された重みを参照):
for edge in G.edges(data=True):
    u, v, data = edge

    is_defense_u = (DEFENSE_START_ID <= u <= DEFENSE_END_ID)
    is_defense_v = (DEFENSE_START_ID <= v <= DEFENSE_END_ID)

    if is_defense_u and is_defense_v:
        defense_edges.append((u, v))
        defense_widths.append(data['weight'])

# グラフの描画
plt.figure(figsize=(12, 8))
plt.title("GAT Attention: Defense Team Internal Collaboration (Blue-Blue Edges)")

# ノードは全て描画し、エッジのみをフィルタリング
nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, label=True)
nx.draw_networkx_edges(G, pos, edgelist=defense_edges, width=defense_widths, alpha=0.9, edge_color='darkblue')
nx.draw_networkx_labels(G, pos, font_size=8)

plt.gca().set_aspect('equal', adjustable='box')
plt.show()

In [ ]:
# --- 攻撃側のみの内部連携 (Red-Red) ---

# ノードIDの範囲を定義 (攻撃チームはID 0から10と仮定)
ATTACK_START_ID = 0
ATTACK_END_ID = 10

# 攻撃チームのエッジのみをフィルタリングするためのリスト
attack_edges = []
attack_widths = []

# グラフGに格納されているエッジと重みを安全にループ処理
for u, v, data in G.edges(data=True):

    # 接続元と接続先が両方とも攻撃チームの範囲内にあることを確認
    is_attack_u = (ATTACK_START_ID <= u <= ATTACK_END_ID)
    is_attack_v = (ATTACK_START_ID <= v <= ATTACK_END_ID)

    if is_attack_u and is_attack_v:
        # エッジリストに追加
        attack_edges.append((u, v))
        # 🚨 安全に重み (太さ) を取得
        attack_widths.append(data['weight'])

# グラフの描画
plt.figure(figsize=(12, 8))
plt.title("GAT Attention: Attack Team Internal Collaboration (Red-Red Edges)")

# ノードは全て描画し、エッジのみをフィルタリング
# (pos, node_sizes, node_colors, G は前のステップで定義されたものを使用)
nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, label=True)
nx.draw_networkx_edges(G, pos, edgelist=attack_edges, width=attack_widths, alpha=0.9, edge_color='darkred')
nx.draw_networkx_labels(G, pos, font_size=8)

plt.gca().set_aspect('equal', adjustable='box')
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.show()

print("✅ 攻撃側のみの連携グラフ描画が完了しました。")

🛡️ 守備側のみの連携 (Blue-Blue Edges)グラフimage_6ec271.png (守備側のみ)傾向中央から右側にかけて、守備側選手（青）同士の太いエッジが多数確認される (例: 13-16, 17-21)。分析守備側の組織化: モデルは、この決定的な瞬間に守備側がマークの受け渡しやカバーリング（カバーディフェンス）を行うための内部的な連携に強く注目しています。これは、守備側が数的不利の中で、なんとか組織を維持しようとしている行動が、予測に重要であると判断されたことを示します。結論守備側は諦めておらず、組織を維持しようとする協調性が、予測の重要な情報源となっている。3. ⚔️ 攻撃側のみの連携 (Red-Red Edges)グラフimage_6ec255.png (攻撃側のみ)傾向グラフ全体（特にピッチ中央から右にかけて）にわたり、太く長いエッジが広範囲に確認される (例: 10-6, 7-6)。分析攻撃側の空間支配: ボール周辺だけでなく、**遠くの選手（ノード 10, 7）**とも太いエッジで結ばれています。これは、攻撃側がピッチの深さと幅を広く使い、守備側にタテ・ヨコに広いエリアを守らせることで、守備を分断しようとする戦術にモデルが注目していることを示します。結論成功は、局所的な連携だけでなく、遠くのサポートランやスペースを広げる動きといった、広範な攻撃側の協調性によって支えられている。